In [651]:
import pandas as pd
import pickle as pkl
import string
import nltk
import numpy as np; np.random.seed(7)
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity

## Data Loadings

### word 2 vec repre

In [85]:
w2v_map = {}
with open('data/vectors_pruned.200.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        wv = line.strip().split(' ')
        word = wv.pop(0)
        w2v_map[word] = np.array(list(map(float, wv)))

In [99]:
w2i_map = {}
for i, key in enumerate(w2v_map.keys()):
    w2i_map[key] = i

In [101]:
pkl.dump(w2i_map, open('data/word_idx_map.pkl','wb'))

### map Q idx to context

In [96]:
w2v_matrix = np.zeros(( len((w2v_map.keys())), 200 ))
counter = 0
for _, val in w2v_map.items():
    w2v_matrix[counter] = val
    counter += 1

In [102]:
pkl.dump(w2i_map, open('data/w2v_matrix.pkl','wb'))

In [247]:
def w2v(w):
    return w2v_matrix[w2i_map[w]]

In [452]:
def sen2w(sen):
    processed = []
    sen = re.sub(r'[!#\'(),/:?\{}]', ' ', sen).strip().split()
    if len(sen) > 100:
        sen = sen[:100]
    for w in sen:
        # ignore date
        if re.match(r'\d{1,}-\d{1,}-\d{1,}', w):
            continue
        if re.match(r'\d{1,}:\d{1,}', w):
            continue
        if w in w2i_map:
            processed += [w]
        else:
            separated = re.findall(r"[^\W\d_]+|\d+|[=`%$\^\-@;\[&_*>\].<~|+\d+]", w)
            for separate_w in separated:
                if separate_w in w2i_map:
                    processed += [separate_w]
    return processed

In [468]:
context_repre = {}
with open('data/text_tokenized.txt', 'r') as src:
    src = src.read().strip().split('\n')
    for line in src:
        context = line.strip().split('\t')
        qid = context.pop(0)
        if len(context) == 1:
            context_repre[int(qid)] = {'t': sen2w(context[0]), 'b': None}
        else:
            context_repre[int(qid)] = {'t':sen2w(context[0]), 'b': sen2w(context[1])}

### pair building --- (q, q+), (q, q-)

In [61]:
train_df = pd.read_csv('data/train_random.txt', header=None, delimiter='\t', names=['Q','Q+','Q-'])

In [35]:
train_df.head()

,Q,Q+,Q-
0,262144,211039,227387 413633 113297 356390 256881 145638 2962...
1,491522,65911,155119 402211 310669 383107 131731 299465 1633...
2,240299,168608 390642,368007 70009 48077 376760 438005 228888 142340...
3,196614,205184,334471 163710 376791 441664 159963 406360 4300...
4,360457,321532,151863 501857 217578 470017 125838 31836 42066...


In [128]:
def build_set_pair_with_idx(df):
    idx_set = {}
    for idx, row in df.iterrows():
        idx_set[row['Q']] = {'pos': np.array(list(map(int, row['Q+'].split(' ')))), \
                             'neg': np.array(list(map(int, row['Q-'].split(' '))))}
    return idx_set

In [129]:
train_idx_set = build_set_pair_with_idx(train_df)

## Process Batch

In [387]:
def contxt2vec(title, body=None):
    
    if body == None:
        body = []
    
    v = np.zeros( (len(title) + len(body), 200) )
    counter = 0
    
    for t in title:
        v[counter] = w2v(t)
        counter += 1

    for b in body:
        v[counter] = w2v(b)
        counter += 1
    
    return v

In [669]:
def process_batch(qids, idx_set):

    batch_x = []
    batch_y = np.zeros((len(qids) * 20, 1))
    counter = 0
    
    max_len = 0
    for qid in qids:
        
        qid_v = contxt2vec(context_repre[qid]['t'], context_repre[qid]['b'])
        
        q_pos = idx_set[qid]['pos']
        
        # usually one sample
        for qid_pos in q_pos:
            #if counter == batch_size: break
            title, body = context_repre[qid_pos]['t'], context_repre[qid_pos]['b']
            max_len = max(max_len, len(title) + len(body))
            batch_x += [(qid_v, contxt2vec(title, body))]
            batch_y[counter] = 1
            counter += 1
        
        q_neg = idx_set[qid]['neg']
        q_neg_sample_indices = np.random.choice(range(100), size=20 - len(q_pos))
        q_random_neg = q_neg[q_neg_sample_indices]
        
        for qid_neg in q_random_neg:
            #if counter == batch_size: break
            title, body = context_repre[qid_neg]['t'], context_repre[qid_neg]['b']
            max_len = max(max_len, len(title) + len(body))
            batch_x += [(qid_v, contxt2vec(title, body))]
            batch_y[counter] = 0
            counter += 1

    # padding
    seq_len = []
    padded_batch_x = np.zeros(( len(qids) * 20, max_len, 400 ))
    for i, (q, q_) in enumerate(batch_x):
        seq_len += [ (q.shape[0], q_.shape[0]) ]
        padded_batch_x[i,:q.shape[0],:200] = q
        padded_batch_x[i,:q_.shape[0],200:] = q_
    
    return padded_batch_x, batch_y, seq_len

In [645]:
qids = list(train_idx_set.keys())[:10]

In [646]:
batch_x, batch_y, seq_len = process_batch(qids, train_idx_set)

## Models

In [507]:
class EmbeddingLayer(nn.Module):
    
    def __init__(self, input_size, hidden_size, layer_type):
        
        super(EmbeddingLayer, self).__init__()
        
        self.hidden_size = hidden_size
        
        if layer_type == 'lstm':
            self.embedding_layer = nn.LSTM(input_size, hidden_size)
        elif layer_type == 'cnn':
            pass
        
        self.tanh = nn.Tanh()
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (Variable(torch.zeros(1, 1, self.hidden_size)), Variable(torch.zeros(1, 1, self.hidden_size)))

    def forward(self, context):
        lstm_out, self.hidden = self.embedding_layer(context, (self.tanh(self.hidden[0]), self.tanh(self.hidden[1])))
        return lstm_out

In [668]:
def cos_sim(qv, qv_):
    return torch.dot(qv, qv_, dim=0) / ( torch.sqrt(torch.sum(qv ** 2, dim=0)) * torch.sqrt(torch.sum(qv_ ** 2, dim=0) ) )

In [673]:
def loss(qv, qv_, y):
    cos_scores = cos_sim(qv, qv_)
    pos_score = cos_scores * y
    neg_score = torch.max(cos_scores * (1 - y))
    diff = (neg_score - pos_score) * y 
    loss = (dff * ( diff > 0 )) / torch.sum(y)
    return loss

In [674]:
embedding_layer = EmbeddingLayer(200, 240, 'lstm')
q = Variable(torch.FloatTensor(batch_x[:,:,:200]))
q_ = Variable(torch.FloatTensor(batch_x[:,:,200:]))